## Constructing a basis set (occupational DoF)

### Create a CASM project

This example will begin with the same ZrO project used in the [Defining the "prim"](https://prisms-center.github.io/CASMcode_docs/pages/tutorials/#defining-the-prim) tutorial.

In [ ]:
prim_str='
{
  "basis" : [
    {
      "coordinate" : [ 0.0000000, 0.0000000, 0.0000000 ],
      "occupants" : [ "Zr" ]
    },
    {
      "coordinate" : [ 0.6666666667, 0.3333333333, 0.5000000 ],
      "occupants" : [ "Zr" ]
    },
    {
      "coordinate" : [ 0.3333333333, 0.6666666667, 0.2500000 ],
      "occupants" : [ "Va", "O" ]
    },
    {
      "coordinate" : [ 0.3333333333, 0.6666666667, 0.7500000 ],
      "occupants" : [ "Va", "O" ]
    }
  ],
  "coordinate_mode" : "Fractional",
  "description" : "hcp Zr with octahedral interstitial O ",
  "lattice_vectors" : [
    [ 3.23398686, 0.00000000, 0.00000000 ],
    [ -1.61699343, 2.80071477, 0.00000000 ],
    [ -0.00000000, 0.00000000, 5.16867834 ]
  ],
  "title" : "ZrO"
}'

In [ ]:
start=$(pwd)
mkdir -p $start/bset/ZrO && cd $start/bset/ZrO

echo "$prim_str" > prim.json
ccasm init

### The basis set specifications file (bspecs.json)

A cluster expansion basis set is specified using:
- [Basis Set Specs](https://prisms-center.github.io/CASMcode_docs/formats/casm/clex/ClexBasisSpecs/): Specifications, written in a "bspecs.json" file, consisting of two components:
  - [Basis Function Specs](https://prisms-center.github.io/CASMcode_docs/formats/casm/basis_set/BasisFunctionSpecs/): Control the type and order of basis functions 
  - [Cluster Specs](https://prisms-center.github.io/CASMcode_docs/formats/casm/clusterography/ClusterSpecs/): Control which cluster basis functions are generated


By default, the "bspecs.json" file is written to:

    <root>/basis_sets/bset.default/bspecs.json
    
Where:
- `<root>`: indicates the CASM project root directory
- `bset.default`: indicates the choice of basis set
  - The is a directory, prefixed with `bset.`, that allows CASM to track multiple basis sets in a single project.
  - Use `casm settings` for creating additional basis sets and switching the current choice of basis set

### Example "bspecs.json"

In [ ]:
bspecs_str='{
  "basis_function_specs" : {
    "global_max_poly_order": 3,
    "dof_specs": {
      "occ": {
        "site_basis_functions" : "occupation"
      }
    }
  },
  "cluster_specs": {
    "method": "periodic_max_length",
    "params": {
      "orbit_branch_specs": {
        "2" : {"max_length" : 6.},
        "3" : {"max_length" : 3.}
      }
    }
  }
}'

In [ ]:
# create the bspecs.json file
echo "$bspecs_str" > basis_sets/bset.default/bspecs.json

# print the prim.json file contents
cat basis_sets/bset.default/bspecs.json

### Generate the basis set

The `-u,--update` command:
- Generates the basis set based on the current "bspecs.json" file
- Writes C++ source code
- Compiles the source code
  - For large basis sets this may be slow, but it only has to be done once
  - Use `--no-compile` to get basis set information without waiting for compilation
- Use `-f,--force` to update and overwrite any existing generated files 


In [ ]:
ccasm bset -u

### Print basis set properties - Basis functions

Use `bset --functions` to print basis functions
- This prints the basis functions for easy viewing
  - It only prints the basis functions associated with the prototype cluster
- Use `--align` to print nicely formatted formulas for Latex
- The underlying data can be found in:
  - The `basis.json` file:
  
        <root>/basis_sets/bset.default/basis.json
        
  - The generated source code for the "clexulator" (cluster expansion calculator):
  
        <root>/basis_sets/bset.default/ZrO_clexulator.cc
- Use `--print-invariant-group` to print the symmetry operations that leave the prototype cluster function invariant


**Notes:**
- Occupation site basis functions are printed as:
      \phi_{b,i}(s_{j})
  where:
  - `b`: is the sublattice index
  - `i`: is the site basis function index
  - `s_{n}`: is the occupation value on the `j`-th site in the cluster (not the entire configurationn)
- The constant site basis function is implicitly included only:
  - A site with two allowed occupants will show as having 1 site basis function, with index 0
  - A site with three allowed occupants will show as having 2 site basis functions, with indices [0, 1]
  - etc.

In [ ]:
# print basis functions
ccasm bset --functions

### Print basis set properties - Clusters and cluster orbits

- Use `--orbits` to print the generated cluster orbits for easy viewing
- Use `--clusters` to print the generated clusters in each orbit
- The underlying data can be found in:

      <root>/basis_sets/bset.default/clust.json
      
- Use `--print-invariant-group` to print the symmetry operations that leave the prototype cluster invariant
- Use `--print-equivalence-map` with `--clusters` to print the symmetry operations that map prototype cluster onto an equivalent cluster

In [ ]:
# print cluster orbits
ccasm bset --clusters --print-invariant-group --print-equivalence-map

In [ ]:
# show the beginning of the clust.json file 
head basis_sets/bset.default/clust.json

### Evaluate correlations

Use `casm select` and `casm query` to selection configuration in the CASM database and query correlations.

In [ ]:
# First, generate configurations to query
# This enumerates unique configurations up to volume 4 unit cells
ccasm enum -m ConfigEnumAllOccupations --max 4

In [ ]:
# Select all configurations, and write the correlation matrix
ccasm select --set-on

In [ ]:
# Query correlations ('corr') and output to corr.txt
ccasm query -k corr -o corr.txt

In [ ]:
# View the beginning of the file containing the correlation matrix
head corr.txt

Clean up tutorial data:

In [ ]:
cd $start && rm -r $start/bset